In [ ]:
%%writefile .env

OPENAI_API_KEY = "my_api_key"

Writing .env


In [ ]:
!pip install python-dotenv
!pip install openai

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
from openai import OpenAI


In [ ]:
client = OpenAI(api_key=openai.api_key)  # or use environment variable


In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)

# Inspect the full message
print(response.choices[0].message)

# If function_call exists, handle it
if response.choices[0].message.function_call:
    print("Function call detected:")
    print(response.choices[0].message.function_call)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather'), tool_calls=None)
Function call detected:
FunctionCall(arguments='{"location":"Boston","unit":"celsius"}', name='get_current_weather')


In [ ]:
function_name = response.choices[0].message.function_call.name
arguments = json.loads(response.choices[0].message.function_call.arguments)

if function_name == "get_current_weather":
    function_response = get_current_weather(
        location=arguments.get("location"),
        unit=arguments.get("unit", "fahrenheit")
    )

    print("Function Response:", function_response)


Function Response: {"location": "Boston", "temperature": "72", "unit": "celsius", "forecast": ["sunny", "windy"]}
